In [1]:
import os

In [2]:
%pwd

'f:\\DS MLOps\\dsproject\\research'

In [3]:
os.chdir("f:/DS MLOps/dsproject")

In [4]:
%pwd

'f:\\DS MLOps\\dsproject'

In [5]:
import pandas as pd
df = pd.read_csv("artifacts/data_ingestion/unzip/winequality-red.csv")

In [6]:
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.dsproject.constants import * #CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.dsproject.utils.common import read_yaml, create_directories

In [10]:
class DataConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config['artifacts_root']])

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=config.status_file,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config
        
    
        

In [11]:
import os
from src.dsproject import logger

In [14]:
# Data Validation Component
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self)-> bool:
        try:
            validation_status= None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_columns=list(data.columns)
            all_schema=self.config.all_schema.keys()

            for col in all_columns:
                if col not in all_schema:
                    validation_status= False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status : {validation_status}")
                else:
                    validation_status= True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status : {validation_status}")

            return validation_status
        except Exception as e:
            raise e

            


In [15]:
try:
    config=DataConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    
    logger.info("Data Validation completed successfully.")
except Exception as e:
    raise e

[2025-11-12 15:01:31,133: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-11-12 15:01:31,135: INFO: common: YAML file: params.yaml loaded successfully]
[2025-11-12 15:01:31,138: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-11-12 15:01:31,140: INFO: common: Directory created at: artifacts]
[2025-11-12 15:01:31,141: INFO: common: Directory created at: artifacts/data_validation]
[2025-11-12 15:01:31,150: INFO: 1652930534: Data Validation completed successfully.]
